### Análise Explorátoria inicial dos Dados  

>  Quais bases são mais interessantes para se usar no projeto? <br>

> Quais as variáveis mais interessantes a serem análisadas? <br>

> Qual a tipagem dos dados? <br>

> Qual a distribuição dessas variáveis? <br>

In [1]:
#Importando as bibliotecas 
import pandas as pd
import numpy as np

In [2]:
#lendo as bases de dados  
notas_ficais = pd.read_csv('../data/raw/201904 sales reciepts.csv')
clientes = pd.read_csv('../data/raw/customer.csv')

#Fazendo uma cópia para a segurança 
nf = notas_ficais.copy()
cl = clientes.copy()

In [3]:
#Visualizando a tabela de notas fiscais 
nf.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [4]:
#Verificando as informações gerais da tabela de notas fiscais 
nf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49894 entries, 0 to 49893
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    49894 non-null  int64  
 1   transaction_date  49894 non-null  object 
 2   transaction_time  49894 non-null  object 
 3   sales_outlet_id   49894 non-null  int64  
 4   staff_id          49894 non-null  int64  
 5   customer_id       49894 non-null  int64  
 6   instore_yn        49894 non-null  object 
 7   order             49894 non-null  int64  
 8   line_item_id      49894 non-null  int64  
 9   product_id        49894 non-null  int64  
 10  quantity          49894 non-null  int64  
 11  line_item_amount  49894 non-null  float64
 12  unit_price        49894 non-null  float64
 13  promo_item_yn     49894 non-null  object 
dtypes: float64(2), int64(8), object(4)
memory usage: 5.3+ MB


In [5]:
#Renomeando as colunas 
nf = nf.rename(columns={'transaction_id':'id_transacao', 
                        'transaction_date':'data_transacao', 
                        'transaction_time':'hora_transacao',
                        'sales_outlet_id':'id_loja', 
                        'staff_id':'id_funcionario', 
                        'customer_id':'id_cliente', 
                        'product_id':'id_produto', 
                        'quantity':'quantidade', 
                        'line_item_amount':'preco_total',
                        'unit_price':'preco_unitario', 
                        'promo_item_yn':'item_promocional'})

In [6]:
#Juntando as colunas data_transacao e hora_transacao para completar a informação do datetime em uma só coluna 
nf['data_hora_transacao'] = nf.apply(lambda x: pd.to_datetime(x['data_transacao'] + ' ' + x['hora_transacao'], format='%Y-%m-%d %H:%M:%S'), axis=1)

In [7]:
#Removendo as colunas "data" e "hora" originais
nf = nf.drop(['data_transacao', 'hora_transacao'], axis=1)

#Trocando a posição das colunas 
nf = nf.reindex(columns=['id_transacao', 'data_hora_transacao','id_loja', 'id_funcionario', 'id_cliente', 'instore_yn',
       'order', 'line_item_id', 'id_produto', 'quantidade', 'preco_total',
       'preco_unitario', 'item_promocional'])

In [8]:
#Verificando o periodo de datas
nf['data_hora_transacao']

0       2019-04-01 12:04:43
1       2019-04-01 15:54:39
2       2019-04-01 14:34:59
3       2019-04-01 16:06:04
4       2019-04-01 19:18:37
                ...        
49889   2019-04-29 16:51:58
49890   2019-04-29 16:51:14
49891   2019-04-29 11:17:36
49892   2019-04-29 15:45:52
49893   2019-04-29 15:45:52
Name: data_hora_transacao, Length: 49894, dtype: datetime64[ns]

In [17]:
#Verificando se temos dados duplicados na tabela de nf
nf.duplicated().value_counts()


False    49894
dtype: int64

In [19]:
#Verificando o número exclusivo(unico) de clientes que fizeram pedidos 
nf.id_cliente.unique().shape

(2248,)

In [20]:
#Verificação de clientes que fizeram mais de um pedido 
49894 - nf.id_cliente.unique().shape[0]

47646

In [21]:
#Número de pedidos por loja 
nf.id_loja.value_counts()

8    17071
3    16829
5    15994
Name: id_loja, dtype: int64

In [32]:
#Describe de var númericas 
nf.describe()

,id_transacao,id_loja,id_funcionario,id_cliente,order,line_item_id,id_produto,quantidade,preco_total,preco_unitario
count,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000,49894.000000
mean,869.056059,5.351846,25.359582,2282.324468,1.173428,1.631860,47.878983,1.438209,4.682646,3.384645
std,857.863149,2.074796,12.466490,3240.551757,1.025445,1.412881,17.928355,0.543039,4.436668,2.682545
min,1.000000,3.000000,6.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.800000
25%,223.000000,3.000000,15.000000,0.000000,1.000000,1.000000,33.000000,1.000000,3.000000,2.500000
50%,481.000000,5.000000,26.000000,0.000000,1.000000,1.000000,47.000000,1.000000,3.750000,3.000000
75%,1401.000000,8.000000,41.000000,5412.000000,1.000000,1.000000,60.000000,2.000000,6.000000,3.750000
max,4203.000000,8.000000,45.000000,8501.000000,9.000000,12.000000,87.000000,8.000000,360.000000,45.000000


In [28]:
nf.order.value_counts()

1    48387
7      916
9      253
6      132
2       83
5       61
3       56
8        4
4        2
Name: order, dtype: int64

In [31]:
nf.line_item_id.value_counts()

1     39949
5      6332
2      2547
3       503
6       324
4       177
9        55
8         4
12        1
10        1
7         1
Name: line_item_id, dtype: int64

In [29]:
#Contagem de quantos pedidos por produto
nf.id_produto.value_counts()

71    1034
50    1015
59    1000
61     995
38     993
      ... 
14      48
4       47
12      45
18      42
19      36
Name: id_produto, Length: 80, dtype: int64

In [10]:
#Visualizando a base de dados de clientes 
cl.head()

,customer_id,home_store,customer_first-name,customer_email,customer_since,loyalty_card_number,birthdate,gender,birth_year
0,1,3,Kelly Key,Venus@adipiscing.edu,2017-01-04,908-424-2890,1950-05-29,M,1950
1,2,3,Clark Schroeder,Nora@fames.gov,2017-01-07,032-732-6308,1950-07-30,M,1950
2,3,3,Elvis Cardenas,Brianna@tellus.edu,2017-01-10,459-375-9187,1950-09-30,M,1950
3,4,3,Rafael Estes,Ina@non.gov,2017-01-13,576-640-9226,1950-12-01,M,1950
4,5,3,Colin Lynn,Dale@Integer.com,2017-01-15,344-674-6569,1951-02-01,M,1951


In [12]:
#Verificando as informações gerais da tabela de clientes 
cl.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   customer_id          2246 non-null   int64 
 1   home_store           2246 non-null   int64 
 2   customer_first-name  2246 non-null   object
 3   customer_email       2246 non-null   object
 4   customer_since       2246 non-null   object
 5   loyalty_card_number  2246 non-null   object
 6   birthdate            2246 non-null   object
 7   gender               2246 non-null   object
 8   birth_year           2246 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 158.0+ KB


In [13]:
#Renomeando as colunas 
cl = cl.rename(columns={'customer_id':'id_cliente', 
                        'customer_first-name':'primeiro_nome_cliente', 
                        'customer_email':'email_cliente',
                        'customer_since':'cliente_desde', 
                        'loyalty_card_number':'ncartao_fidelidade', 
                        'birthdate':'data_aniversario', 
                        'gender':'genero',
                        'birth_year':'ano_nascimento'})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id_cliente             2246 non-null   int64 
 1   home_store             2246 non-null   int64 
 2   primeiro_nome_cliente  2246 non-null   object
 3   email_cliente          2246 non-null   object
 4   cliente_desde          2246 non-null   object
 5   ncartao_fidelidade     2246 non-null   object
 6   data_aniversario       2246 non-null   object
 7   genero                 2246 non-null   object
 8   ano_nascimento         2246 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 158.0+ KB


In [14]:
#Transformando data_aniversario, cliente desde em datetime 
def transforme_datetime(df, columns):
    for col in columns:
        df[col] = pd.to_datetime(df[col])

transforme_datetime(cl, ['data_aniversario', 'cliente_desde'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2246 entries, 0 to 2245
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id_cliente             2246 non-null   int64         
 1   home_store             2246 non-null   int64         
 2   primeiro_nome_cliente  2246 non-null   object        
 3   email_cliente          2246 non-null   object        
 4   cliente_desde          2246 non-null   datetime64[ns]
 5   ncartao_fidelidade     2246 non-null   object        
 6   data_aniversario       2246 non-null   datetime64[ns]
 7   genero                 2246 non-null   object        
 8   ano_nascimento         2246 non-null   int64         
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 158.0+ KB


In [18]:
#Verificando se temos dados duplicados na tabela de cl
cl.duplicated().value_counts()

False    2246
dtype: int64

In [23]:
#Verificando o genero dos clientes 
cl.genero.value_counts()

F    977
M    726
N    543
Name: genero, dtype: int64

In [25]:
generos_vazios = cl.loc[cl['genero'] == 'N'] 
print(generos_vazios.head())
print(generos_vazios.shape)

     id_cliente  home_store primeiro_nome_cliente         email_cliente  \
700         701           3       Donovan Trevino      Miranda@eget.com   
701         702           3        Michael Kinney       Samson@diam.com   
702         703           3         Melvin Valdez  Odette@fermentum.gov   
703         704           3       Bernard Everett    Jelani@egestas.edu   
704         705           3         Phillip Lopez    Paki@consequat.org   

    cliente_desde ncartao_fidelidade data_aniversario genero  ano_nascimento  
700    2017-01-10       127-289-9961       1950-09-30      N            1950  
701    2017-01-18       721-800-7676       1951-04-04      N            1951  
702    2017-01-26       916-386-4370       1951-10-08      N            1951  
703    2017-02-04       711-278-7766       1952-04-11      N            1952  
704    2017-02-12       644-583-4372       1952-10-15      N            1952  
(543, 9)
